## Exploring the numeric dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [1]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-'

#### Install the autolabel library

In [2]:
!pip install 'refuel-autolabel[openai]'

#### Download the dataset

In [2]:
from autolabel import get_data

get_data('numeric')

This downloads two datasets:
* `test.csv`: This is the larger dataset we are trying to label using LLMs
* `seed.csv`: This is a small dataset where we already have human-provided labels

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [2]:
import json

from autolabel import LabelingAgent

In [3]:
# load the config
with open('config_numeric.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `named_entity_recognition` (since it's a named entity recognition task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are an expert at extracting Person, Organization, Location, and Miscellaneous entities...` (how we describe the task to the LLM)
* `prompt.labels`: `[
            "Location",
            "Organization",
            "Person",
            "Miscellaneous"
        ]` (the full list of labels to choose from)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [4]:
config

{'task_name': 'ExtractNumericSubstring',
 'task_type': 'named_entity_recognition',
 'dataset': {'label_column': 'CategorizedLabels',
  'text_column': 'example',
  'delimiter': ','},
 'model': {'provider': 'openai', 'name': 'gpt-3.5-turbo'},
 'prompt': {'task_guidelines': 'Extract the last instance of a number being mentioned in the input text. The only category in this task is {labels}',
  'labels': ['Number'],
  'example_template': 'Example: {example}\nOutput:\n{CategorizedLabels}',
  'few_shot_examples': 'seed.csv',
  'few_shot_selection': 'semantic_similarity',
  'few_shot_num': 7}}

In [5]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [6]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $2.3915 │
│ Number of Examples       │ 1000    │
│ Average cost per example │ $0.0024 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

Extract the last instance of a number being mentioned in the input text. The only category in this task is Number

You will return the answer in CSV format, with two columns seperated by the % character. First column is the 
extracted entity and second column is the category. Rows in the CSV are separated by new line character.

Some examples with their output answers are provided below:

Example: That one is okay .
Output:
one%Number

Example: Three other people .
Output:
Three%Number

Example: How 'd you come to think that one up ?
Output:
one%Number

Example: He kept right on playing 'till all five was gone .
Output:
five%Number

Example: I only had three .
Output:
three%Number

Example: No one even seemed to know who was doing the raids .
Output:
one%Number

Example: I could n't hear myself thinking with this one around .
Output:
one%Number

Now I want you to label the following example:
Example: One person thought it was still a good idea .
Output:

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
# now, do the actual labeling
ds = agent.run(ds, max_items=1000)

2023-10-03 00:45:55 autolabel.labeler INFO: Task run already exists.


There is an existing task with following details: id='293988831' created_at=datetime.datetime(2023, 10, 3, 0, 21, 
31, 409626) task_id='de84e44249cd3748acf02cf84973cf32' dataset_id='403b2718ee1791610570520c29197a1a' 
current_index=314 output_file='ExtractNumericSubstring_labeled.csv' status=<TaskStatus.ACTIVE: 'active'> error=None
metrics=None

Evaluating the existing task...

┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ support ┃ completion_rate ┃ f1_exact ┃ f1_strict ┃ f1_partial ┃ f1_ent_type ┃ accuracy ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ 313     │ 1.0             │ 0.8376   │ 0.8376    │ 0.8583     │ 0.879       │ 0.8403   │
└─────────┴─────────────────┴──────────┴───────────┴────────────┴─────────────┴──────────┘

314 examples labeled so far.

Do you want to resume the task? [y/n]:

 n


Deleted the existing task and starting a new one...

Output()

2023-10-03 00:47:34 openai INFO: error_code=rate_limit_exceeded error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89070 / min. Contact us through our help center at help.openai.com if you continue to have issues.' error_param=None error_type=tokens message='OpenAI API error received' stream_error=False
2023-10-03 00:47:34 langchain.chat_models.openai WARNING: Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkYhAIYGmLcxLmarMmAPo on tokens per min. Limit: 90000 / min. Current: 89070 / min. Contact us through our help center at help.openai.com if you continue to have issues..
2023-10-03 00:47:35 openai INFO: error_code=rate_limit_exceeded error_message='Rate limit reached for default-gpt-3.5-turbo in organization org-etZVkY